In [1]:
import pandas as pd
import sys
import os
import numpy as np
import glob
#from Bio.pairwise2 import format_alignment
#from Bio import pairwise2
#import dask.dataframe as dd
#from dask.multiprocessing import get
#import swifter
from multiprocessing import  Pool
import functionsv2 as functions
import Levenshtein as lv

In [2]:
bridge='TCTTCAGCGTTCCCGAGA'

In [3]:
data = 'MouseLiver/'

# Get sequenced bead barcodes and positions

Load barcode file from Broad

In [4]:
barcodes=[]
with open('U:/smc/public/SMC/SlideSeq/Pucks/20210218_EvanBatch1/210113_17/BeadBarcodes.txt', 'r') as fh:
    lines = []
    for f in fh:
        filteredChars = ''.join((filter(lambda x: x not in [','], f.rstrip())))
        barcodes.append(filteredChars)
barcodes=np.asarray(barcodes)

Load location file from Broad

In [5]:
positions=[]
with open('U:/smc/public/SMC/SlideSeq/Pucks/20210218_EvanBatch1/210113_17/BeadLocations.txt', 'r') as fh:
    lines = []
    for f in fh:
        positions.append(f.rstrip().split(','))
positions=np.asarray(positions)

In [6]:
mapper=np.append(positions, np.reshape(barcodes, [1,barcodes.shape[0]]), axis=0)
mapper=np.swapaxes(mapper, 0,1)

In [7]:
df=pd.DataFrame(mapper, columns=['X', 'Y', 'Barcode'])

In [8]:
df['BridgedBarcode']=df['Barcode'].map(lambda x: x[0:8]+bridge+x[8:])

In [9]:
df.iloc[56591]

X                                           3487.1
Y                                           2269.8
Barcode                             GCAGCGAACCTCAC
BridgedBarcode    GCAGCGAATCTTCAGCGTTCCCGAGACCTCAC
Name: 56591, dtype: object

# Get all NGS sequences

In [10]:
def process(lines=None):
    ks = ['name', 'sequence', 'optional', 'quality']
    return {k: v for k, v in zip(ks, lines)}

In [11]:
n = 4

lst=[]
cur=0
A=[]
B=[]
C=[]
Names=[]
Optionals=[]
Qualities=[]
C_UMI=[]
bridged=0
files=['n_1_1_AGGCAGAA.fastq', 'n_2_1_AGGCAGAA.fastq', 'n_3_1_AGGCAGAA.fastq', 'n_4_1_AGGCAGAA.fastq' ]
#for f in glob.glob('*1_G*.fastq'):
for f in files:
    with open(data+f, 'r') as fh:
        print(f)
        lines = []
        for line in fh:
            cur=cur+1
            if (cur%10000000==0):
                print(cur)
            lines.append(line.rstrip())
            if len(lines) == n:
                record = process(lines)
                lst.append(len(record['sequence']))
                Names.append(record['name'])
                Optionals.append(record['optional'])
                Qualities.append(record['quality'])
                C.append(record['sequence'][0:32])
                C_UMI.append(record['sequence'][32:])
                lines = []

n_1_1_AGGCAGAA.fastq
10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
100000000
110000000
120000000
130000000
n_2_1_AGGCAGAA.fastq
140000000
150000000
160000000
170000000
180000000
190000000
200000000
210000000
220000000
230000000
240000000
250000000
n_3_1_AGGCAGAA.fastq
260000000
270000000
280000000
290000000
300000000
310000000
320000000
330000000
340000000
350000000
360000000
370000000
380000000
n_4_1_AGGCAGAA.fastq
390000000
400000000
410000000
420000000
430000000
440000000
450000000
460000000
470000000
480000000
490000000
500000000
510000000


In [12]:
C=np.asarray(C)
C_UMI=np.asarray(C_UMI)
values, inverter, counts = np.unique(C, return_counts=True, return_inverse=True)

In [13]:
pairs=[[s, v] for s,v in zip(values, counts) ]
pairs=np.asarray(pairs)

In [14]:
[pairs.shape, len(C)]

[(12937066, 2), 129113616]

In [15]:
np.save(data+'pairs.npy', pairs)
np.save(data+'reads.npy', C)
np.save(data+'read_inverter.npy', inverter)
np.save(data+'UMI_reads.npy', C_UMI)

In [16]:
np.save(data+'Qualities.npy', np.asarray(Qualities))
np.save(data+'Optionals.npy', np.asarray(Optionals))
np.save(data+'Names.npy', np.asarray(Names))

# Load from existing file

Skip here if have already saved the data

In [10]:
pairs=np.load(data+'pairs.npy')
C=np.load(data+'reads.npy')
inverter=np.load(data+'read_inverter.npy')
C_UMI=np.load(data+'UMI_reads.npy')

# Map spatial reads to barcodes

We have to filter somwhere, 100 seems reasonable.

In [11]:
trimmed_pairs = pairs[pairs[:,1].astype(int)>15]


In [12]:
print(C.shape[0], pairs.shape[0], trimmed_pairs.shape[0], np.sum(trimmed_pairs[:,1].astype(int)))

129113616 12937066 922298 103860864


In [13]:
[trimmed_pairs.shape[0]/pairs.shape[0], np.sum(trimmed_pairs[:,1].astype(int))/np.sum(pairs[:,1].astype(int))]

[0.07129112582404697, 0.8044144933559912]

So by throwing out any spatial reads that don't repeat at least 100 times, we throw out 98% of the uniquely read reads, but only 27% of the total number of reads, can certainly live with this.

This is the long part, at least 2 hours.  It will try to find the index in BridgedBarcode to match each of our featured trimmed_pairs.  If it fails, will give a -1 in that position.

In [14]:
#pool_class=functions.My1stClass()
pool_class=functions.My3rdClass()
pool_class.set_data(df['BridgedBarcode'])
lst=list(trimmed_pairs[:,0])
pool = Pool()
s= (pool.map(pool_class.myfu, (lst)))
s=np.asarray(s)

In [15]:
np.save(data+'s.npy', s)

In [16]:
s = np.load(data+'s.npy')

In [17]:
np.sum(trimmed_pairs[s[:,0]<3][:,1].astype(int))/len(C)

0.08981866017910922

In [18]:
np.sum(trimmed_pairs[s[:,0]<3][:,1].astype(int))/len(C)

0.08981866017910922

Wow, 47% of reads now map to a barcode, some of these might be flimsy or incorrect though.

In [19]:
np.unique(s[s[:,0]<3,1]).shape[0]/len(df)

0.28586666666666666

In [20]:
np.unique(s[s[:,0]<3,1]).shape[0]/len(df)

0.28586666666666666

46% of all beads now have reads mapping to them.  Given that the tissue covers less than half, this seems OK.

In [21]:
master_df = pd.DataFrame(np.concatenate([s, trimmed_pairs], axis=1), columns=['Distance', 'BeadPos', 'Barcode', 'ReadCount'])
master_df['ReadCount'] = master_df['ReadCount'].astype(np.int64)
master_df['Distance'] = master_df['Distance'].astype(np.int64)
master_df['BeadPos'] = master_df['BeadPos'].astype(np.int64)

In [22]:
master_df.to_csv(data+'master_df.csv')

In [23]:
master_df[master_df['Distance']<1]

,Distance,BeadPos,Barcode,ReadCount
2253,0,22893,AAAACGACTCTTCAGCGTTCCCGAGAGATATG,142
51182,0,74178,AATTCCCTTCTTCAGCGTTCCCGAGAAAGTAA,18
74017,0,52078,ACCCAGAATCTTCAGCGTTCCCGAGACTCCCC,46
128037,0,34603,AGATTTAATCTTCAGCGTTCCCGAGAAGCTAG,667
193246,0,26620,ATGAGAACTCTTCAGCGTTCCCGAGAGTTCGG,26
208839,0,66499,ATTCAGCATCTTCAGCGTTCCCGAGAAGGCGA,623
231888,0,71142,CAATCTCGTCTTCAGCGTTCCCGAGAAATCCA,51
252850,0,183,CAGACCGATCTTCAGCGTTCCCGAGACCTTTT,5698
272152,0,12454,CATCTTCATCTTCAGCGTTCCCGAGAACGCCT,31
292779,0,3472,CCAGCGGCTCTTCAGCGTTCCCGAGAACACTT,17


## Aggregate on distance

In [71]:
pivot_df = master_df[master_df['Distance']<=16].pivot_table(index='BeadPos', aggfunc=np.sum, values='ReadCount')
for a in range(15,-1,-1):
    b = master_df[master_df['Distance']<=a].pivot_table(index='BeadPos', aggfunc=np.sum, values='ReadCount')
    pivot_df = pivot_df.merge(b, left_index=True, right_index=True, how='left')
pivot_df = pivot_df.fillna(0)
pivot_df.columns=range(16,-1,-1)  
pivot_df = pivot_df.merge(df, left_index=True, right_index=True)

In [72]:
pivot_df.to_csv(data+'BeadReadsHammingDistanceBad.csv')

# Make images

In [ ]:
import tifffile
import scipy.ndimage as ndimage
import math

In [ ]:
def save_image(X, Y, Counts, fpath):
    img=np.zeros([1+7000, 1+7000])
    img[math.floor(X),math.floor(Y)]=1+Counts
    dimg = ndimage.maximum_filter(img, size=8)
    tifffile.imsave(fpath, dimg)

### Make image of all beads

Use barcodes and positions, have already loaded them

In [ ]:
with open('U:/smc/public/SMC/SlideSeq/Pucks/20210218_EvanBatch1/210113_17/BeadLocations.txt', "r") as myfile:
    aller_beads = myfile.readlines()
aller = np.array([np.array(str(s).split(',')) for s in aller_beads])
aller = aller.astype(float).T    
aller = np.round(aller).astype(int)

In [ ]:
save_image(aller[:,0], aller[:,1], aller[:,0]*0+10, data+'AllBeads.tif')

### Make image of beads with counts

In [ ]:
save_image(aller[:,0], aller[:,1], master_df+10, data+'AllBeads.tif')

### Make image stack of beads with counts and different Levenstein cutoffs

# END USEFUL STUFF

In [208]:
uns, vas, = np.unique(s[s[:,0]<1][:,1], return_counts=True)
(np.histogram(vas, bins=range(0,50,1))[0]/np.sum(np.histogram(vas, bins=range(0,50,1))[0])*100).astype(int)

array([  0, 100,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [209]:
uns, vas, = np.unique(s[s[:,0]<2][:,1], return_counts=True)
(np.histogram(vas, bins=range(0,50,1))[0]/np.sum(np.histogram(vas, bins=range(0,50,1))[0])*100).astype(int)

array([ 0, 76, 13,  4,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [210]:
uns, vas, = np.unique(s[s[:,0]<3][:,1], return_counts=True)
(np.histogram(vas, bins=range(0,50,1))[0]/np.sum(np.histogram(vas, bins=range(0,50,1))[0])*100).astype(int)

array([ 0, 62, 19,  7,  3,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [211]:
uns, vas, = np.unique(s[s[:,0]<4][:,1], return_counts=True)
(np.histogram(vas, bins=range(0,50,1))[0]/np.sum(np.histogram(vas, bins=range(0,50,1))[0])*100).astype(int)

array([ 0, 57, 20,  9,  4,  2,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [199]:
len(vas)

32696

In [164]:
pivot_df['BeadPos']

KeyError: 'BeadPos'

In [76]:
df

,X,Y,Barcode,BridgedBarcode
0,5427.1,2005.8,CTTTGCCCCTTAGN,CTTTGCCCTCTTCAGCGTTCCCGAGACTTAGN
1,3379,3890.6,TGGGCTCTCGTCAN,TGGGCTCTTCTTCAGCGTTCCCGAGACGTCAN
2,3219.1,5260.2,GTCGCTTTAGGNTT,GTCGCTTTTCTTCAGCGTTCCCGAGAAGGNTT
3,1733.5,5529.6,TTTNTTTTTTTTTT,TTTNTTTTTCTTCAGCGTTCCCGAGATTTTTT
4,552.49,3069.7,TTTTTTTTTTTTTT,TTTTTTTTTCTTCAGCGTTCCCGAGATTTTTT
...,...,...,...,...
83343,2168.3,878.12,AAAAANAAAAAAAA,AAAAANAATCTTCAGCGTTCCCGAGAAAAAAA
83344,628.42,577.23,TAANAAAAAAAAAA,TAANAAAATCTTCAGCGTTCCCGAGAAAAAAA
83345,655.85,3646.2,GAANAAAAAAAAAA,GAANAAAATCTTCAGCGTTCCCGAGAAAAAAA
83346,667.13,3591.2,CAANAAAAAAAAAA,CAANAAAATCTTCAGCGTTCCCGAGAAAAAAA


In [83]:
np.where(s>-1)

(array([   525,    535,    538, ..., 212723, 212724, 212761], dtype=int64),)

In [84]:
s[525]

53847

In [85]:
[lst[525], df['BridgedBarcode'].iloc[s[525]]]

['AAAAAAACTCTTCAGCGTTCCCGAGAAAGGAC', 'AAAAAAACTCTTCAGCGTTCCCGAGAATGGCC']

In [72]:
trimmed_pairs[34660,0][0:8] + trimmed_pairs[34660,0][-6:]

'AGCTCCCCGTCACC'

In [73]:
trimmed_pairs[34660]

array(['AGCTCCCCTCTTCAGCGTTCCCGAGAGTCACC', '1173'], dtype='<U32')

In [69]:
df['BridgedBarcode'].iloc[56591:56592]

56591    GGGTCCCTTCTTCAGCGTTCCCGAGAGTCACC
Name: BridgedBarcode, dtype: object

In [57]:
df['BridgedBarcode'].iloc[56000:57000].iloc[s[559]]

'TAAAAAGGTCTTCAGCGTTCCCGAGAACTACC'

In [42]:
trimmed_pairs.shape

(212889, 2)

In [79]:
'NNNNNNNN' + bridge + 'NNNNNN'

'NNNNNNNNTCTTCAGCGTTCCCGAGANNNNNN'

In [56]:
import Levenshtein as lv

In [68]:
lv.distance('CCGAA', 'ACCGA')

2

In [43]:
bridge

'TCTTCAGCGTTCCCGAGA'

In [53]:
df[df['Barcode']=='CGACCCGGGGACAA']

,X,Y,Barcode,BridgedBarcode


In [44]:
C[1000:2000]

array(['GTGCCTGATCTTCACGTTCCCGAGAATAACGG',
       'GGCCTTTATCTTCAGCGTTCCCGAGACTAGTA',
       'GCCTACCTGTTCTGCATTCCCGAGTCCGACTA',
       'ACGTGTTTTTTTTTTTTTTTTTTTTTTTTTTT',
       'ATTCAGCATCTTCAGCGTTCCCGAGATCCTGT',
       'GGCAAAGTCTTCAGCGTTCCCGCCTCGACGAT',
       'TGCGGACTTCTTCAGCGTTCCGAGAATTTACC',
       'GGCTCGGCATCATCATCATCATCCTGGTCGAG',
       'GCTGGAGCTCTTCAGCGTTCCCGAGATTACTC',
       'ATCCTCCATCTGATAACTGAGTTACCAGATCG',
       'AGTTTGAGTCCTCAGCGTTCCCTTTTTTTTTT',
       'GCTACCTCGCCTCGGTTCGGTTTTTTTTTTTT',
       'GACCAACGTCTTCAGCGTTCCCGAGATGAGCA',
       'TTCCGGGTTCTTCAGCGTTCGCCGGGCCGCAA',
       'AACCAGCATCTTCAGCGTTCCCGAGACAAGAT',
       'TTCGCGTTCCCGAGAGTTGGTATCTTCGCAGA',
       'ATGACGAACTGTTCCAGCAGGACAAGGTTCAG',
       'CGACCCGTCTTCAGCGTTCCCGAGAGGGACAA',
       'AGGCGACTCTTCAGCGTTCCCGAAGGCGAAGC',
       'AATGTTTTTTTTTTTTTTTTTTTTTTTTTTTT',
       'TCAGCGAATCTTCAGCGTTCCCGAGACTCGTT',
       'ATCGCCCCTCTTCACCATCTTTCGGGTCTCTA',
       'GTTCTTGCTCGCTGTCGCGTTCGGACCGCATG',
       'GGC

In [23]:
len(df['BridgedBarcode'][0])

32

In [ ]:
56591

In [20]:
len(pairs[0,0])

32

# Filtering and finding

No little piglet, I am afraid that does not work, since shorter goes faster, let's try just looking at how many "apparent" bead barcodes only have a single read corresponding to them

In [14]:
cpairs=pairs[:,1].astype(int)
[np.sum(cpairs[cpairs>1])/np.sum(cpairs), np.sum(cpairs>1)/len(cpairs)]

[0.9692187968859401, 0.4032510600794104]

93% of all reads are contained in the list of barcodes that show up more than once, but 50% of the unique barcodes have only one read, so I think we can safely throw them out since they will add so much to the search time

In [10]:
commonpairmask=pairs[:,1]!='1'

Now let's just look for ones that contain a reasonable bridge sequence

In [11]:
pool_class=functions.MyClass()
pool_class.set_data(bridge)
lst=list(pairs[commonpairmask,0])
pool = Pool()
s= (pool.map(pool_class.myfu, (lst)))
s=np.asarray(s)

In [12]:
bridgedmask=np.zeros(pairs[:,0].shape)
bridgedmask[:]=False
bridgedmask[commonpairmask]=s>14
bridgedmask=bridgedmask.astype(bool)

In [13]:
[np.sum(bridgedmask)/len(s), np.sum(bridgedmask)]

[0.3661221975757859, 3212801]

Surprisingly, only about a third of the filtered sequences contain a bridge sequence with a mismatch or two, but that is still 3M unique sequences, and we only have 74K beads, so there clearly still needs to be a lot of aggregation

In [14]:
dpairs=pairs[bridgedmask,1].astype(int)
np.sum(dpairs)/np.sum(cpairs)

0.6437293744695824

64% of our original reads have survived the '1 read' filter and the 'has bridge' filter.  This is because most of the reads have barcodes that appear many many times, so the list of unique barcodes is almost entirely dominated by barcodes that have 1 or 2 reads.

# Single mismatch, simple way

In [15]:
np.sum(pairs[bridgedmask,1].astype(int)>4)

1374837

If we only look at the top 1M barcodes, this accounts for 59% of all reads, and there are only 64% that have the bridge in them.  There are only 1.3M unique barcodes that have more than 4 reads associated with them, so I will only keep investigating those.

In [16]:
fivepairmask=(pairs[:,1].astype(int)>4) & (bridgedmask) 

In [17]:
[np.sum(fivepairmask)/len(pairs), np.sum(pairs[fivepairmask,1].astype(int))/np.sum(pairs[bridgedmask,1].astype(int)), np.sum(pairs[fivepairmask,1].astype(int))/len(C)]

[0.07844101928131976, 0.942490377785251, 0.6067087413353001]

So we are accounting for only 8% of the unique barcodes found, but 94% of those with a bridge found, and 60% of the total reads.  I think this is acceptable.  This will take about 8 hours on Cyclops.  The output:  s, will be the size of fivepairmask and will have an index corresponding to the ID of the df['BridgedBarcode'] from Broad.  This uses a simple hand made hamming distance calculator, I should have used:  import Levenshtein as lv, lv.distance(a,b).  

Edit, I put in Levenshtein, it takes twice as long but it works beautifully.  Half or more of the mistmatches are in fact shifts, not simple mismatches, so this should collect a LOT more of the reads.

In [49]:
pool_class=functions.My2ndClass()
pool_class.set_data(df['BridgedBarcode'])
lst=list(pairs[fivepairmask,0])
pool = Pool()
s= (pool.map(pool_class.myfu, (lst)))
s=np.asarray(s)

In [50]:
np.sum(fivepairmask)

1374837

Make the spatialmask, which works on the original pairs array.  Also make the final_mapper and position_mapper arrays that are in the same shape as the pairs array so we can use inverter to convert back to the original C array with ALL reads

In [51]:
index_mapper=np.zeros(pairs[:,0].shape)-1
index_mapper[fivepairmask]=s

spatialmask=index_mapper>-1

final_mapper=np.zeros(pairs[:,0].shape).astype('str')
final_mapper[:]=''
final_mapper[spatialmask]=df['BridgedBarcode'][index_mapper[spatialmask].astype(int)]

position_mapper=np.zeros(pairs[:,0].shape).astype('str')
position_mapper[:]=''
position_mapper[spatialmask]=df['X'][index_mapper[spatialmask]]+'_'+df['Y'][index_mapper[spatialmask]]


In [52]:
[np.sum(spatialmask[inverter])/len(C), np.sum(fivepairmask[inverter])/len(C)]

[0.5045388639363898, 0.6067087413353001]

With hamming, this was:
[0.366414527374538, 0.6067087413353001]
Wow!  Got 50% with the Levenshtein instead of hamming code!

In [53]:
np.save('final_mapper.npy', final_mapper)
np.save('position_mapper.npy', position_mapper)
np.save('spatialmask.npy', spatialmask)

In [54]:
helpme=final_mapper[inverter]

In [55]:
C[0:30]

array(['NAGTAAAATCTTCAGCGTTCCCGAGATATAGC',
       'NACAATATCTTCAGCGTTCCCGAGAAGTACGT',
       'NTGTTCGATCTTCAGCGTCCGAGAAATGTGTT',
       'NGCCTATCTCTTCAGCGTTCATCTCTACCTGT',
       'NCCTGGACTCTTTTTTTTTTTTTTTTTTTTTT',
       'NATAACCCTTCAGAAACTTAATACCTCATGAA',
       'NTCATTTTTAGGGTTCATTTGGATTAGTGTTT',
       'NGGATCAGTCTTACGTCGAGAAAGTTCTTCAA',
       'NCAAAGATTCTTCAGCGTTCCCGAGACGACGG',
       'NTCGTGTGTCTTCAGCGTTCCCGAGACCTTAC',
       'NATTTAATTCTTCAGCGTTCCCGAGAAGGCGA',
       'NAGAAGGTCTTCAGCGTTCCCGAGATGCGTTT',
       'NTTGTTGTCTTCAGCGTTCCTAGCACTGATGG',
       'NCCATGATTCTTCAGCGTTCCCGAGAGCCTTT',
       'NCTGGCCTTAGCGTTTTCTTCCTTTGGCTGAA',
       'NCGAAATATCTTCAGCGTTCCCGAGACGACCG',
       'NGGTAGTTTTTTTTTTTTTTTTTTTTTTTTTT',
       'NCCTATGTCTTCAGCGTTCCCGAGGGAAACTT',
       'NGTAGATTTCTTCAGCGTTCCCGAGAGTTCTA',
       'NCTTCGCTTCTTCAACGTTCCCGAGACGTTGT',
       'NCATCATCTCTTCAGCGTTCCCGAGACTTTGA',
       'NTCTGCACTCTTCAGCGTTCCCGAGAGCTTCG',
       'NTTGCGACTCTTCAGCGTTCCCGAAGTTCCTT',
       'NGC

In [56]:
helpme[0:30]

array(['', '', '', '', '', '', '', '', '', '', '', '', '',
       'ACCATGATTCTTCAGCGTTCCCGAGAGCCTTT', '', '', '', '', '', '', '', '',
       '', '', 'TTCTCTGTTCTTCAGCGTTCCCGAGATCTGTC', '', '',
       'TGGGTTCATCTTCAGCGTTCCCGAGACTACGC', '',
       'TTGGAGTTTCTTCAGCGTTCCCGAGACTACCG'], dtype='<U32')

# Deal with UMIs

In [ ]:
C_UMI=np.load('UMI_reads.npy')

helpme is the full, all read list with the corrected spatial barcode OR an empty string.  To get ONLY the ones that are not empty, we take spatialmask[inverter].  From now on, we will only deal with the trimmed full read list (ie ones that have a spatial barcode).

In [57]:
trimmed_helpme=helpme[spatialmask[inverter]]
trimmed_C_UMI=C_UMI[spatialmask[inverter]]

Now we will get out the list of unique Broad-verified spatial barcodes (this is not necessarily ALL of the Broad barcodes, as some probably never have sequences).

In [58]:
fvalues, finverter, fcounts = np.unique(trimmed_helpme, return_counts=True, return_inverse=True)

We already have a hamming code function written that takes a single sequence on the left and a list on the right and finds the one on the right most likely to match the first (-1 if none).

In [59]:
class2=functions.My2ndClass()

In [60]:
trimmed_fixed_C_UMI=np.copy(trimmed_C_UMI)

This function will take a list of sequences, and return a list of the same size but where sparse sequences from the first list will be replaced by a more common sequence if they are within a mismatch or two.

In [61]:
def ReduceRedundancy(source_list):
    rtn_list=np.copy(source_list)
    fvals, finverts, fcts = np.unique(source_list, return_counts=True, return_inverse=True)
    sorted_indices = np.argsort(fcts)
    class2=functions.My2ndClass()
    
    for idx, sidx in enumerate(sorted_indices[0:-1]):
        class2.set_data(fvals[sorted_indices[idx+1:]])
        cand = class2.myfu(fvals[sidx])
        if (cand>-1):
            real_idx=cand+idx+1
            rtn_list[rtn_list==fvals[sidx]]=fvals[sorted_indices[real_idx]]
    return rtn_list

Some day I will parallelize this, but I don't feel like it today...

In [62]:
for idx, f in enumerate(fvalues):
    associated=fvalues==f
    UMIs=(trimmed_C_UMI[(associated)[finverter]])
    fixed_UMIs=ReduceRedundancy(UMIs)
    trimmed_fixed_C_UMI[(associated)[finverter]]=fixed_UMIs
    if (idx%1000==0):
        print(idx)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000


In [363]:
np.where(trimmed_C_UMI!=trimmed_fixed_C_UMI)

(array([      48,      216,     1551, ..., 48691820, 48691852, 48693724],
       dtype=int64),)

In [365]:
[trimmed_C_UMI[48], trimmed_fixed_C_UMI[48]]

['CTCGACAGCA', 'CTCGACAGCC']

In [334]:
tt=(trimmed_C_UMI[(fvalues==fvalues[30])[finverter]])

In [349]:
ttt=ReduceRedundancy(tt)
ttt

[ 1  1  1 10  6  1  1  2 10  2  1  1  1  1  1 11  3  1  1  1  9  1  2  1
  1  6]
[ 0 23 21 19 18 17 14 13 24 11 12  6  5  2  1 10  9  7 22 16  4 25 20  8
  3 15]
Gotone
Gotone
Gotone


array(['ATCGTGGCGT', 'ATCACGGTCT', 'ATCATGAACT', 'TCTGCTAGTT',
       'ATCCCAGCGT', 'ATCAGGCGTG', 'ATCCGCTGTA', 'ATCGATATTT',
       'ATCTTGGCTA', 'ATCTTAGCAA', 'ATCACGGTCT', 'ATCCCAGCGT',
       'ATCGTACCCT', 'ATCGTACCCT', 'ATCACGGTCT', 'ATCGTACCCT',
       'ACTGTCTCTT', 'ATCCCAGCGT', 'TCTGCTAGTT', 'ATCGTGGCGT',
       'ATCCCAGCGT', 'ATCGTGGCGT', 'ATCGTGGCGT', 'ATCCCAGCGT',
       'TCTGCTAGTT', 'ATCGTACCCT', 'ATCACGGTCT', 'ATCCGCTGTA',
       'ATCACGGTCT', 'ATCACGGTCT', 'ATCTCAGTCC', 'ATCAAGCCTG',
       'ATCAGGCGTG', 'TCTGCTAGTT', 'ATCACGGTCT', 'ATCGTCTGTC',
       'CTGTCTCTTA', 'ATCCCAGCGT', 'ATCGTACCCT', 'TCTGCTAGTT',
       'ATCGTGGCGT', 'ATCGTACCCT', 'ATCAGGCGTG', 'ATCATGAACT',
       'ATCGTGGCGT', 'ATCCATACTC', 'ATCGTGGCGT', 'ATCCTTTGCA',
       'ATCCCAGCGT', 'ATCACGGTCT', 'ATCCCAGCGT', 'ATCGTGGCGT',
       'ATCGTACCCT', 'ATCAGGCGTG', 'ATCTTAGCAA', 'ATCCCAGCGT',
       'ATCAGGCGTG', 'TCTGCTAGTT', 'ATCGTACCCT', 'ATCGTACCCT',
       'ATCGTCCACA', 'ATCGTGCCGA', 'ATCGTGGCGT', 'ATCGT

In [347]:
tt

array(['ATCGTGGCGT', 'ATCACGGTCT', 'ATCATGAACT', 'TCTGCTAGTT',
       'ATCCCAGCGT', 'ATCAGGCGTG', 'ATCCGCTGTA', 'ATCGATATTT',
       'ATCTTGGCTA', 'ATCTTAGCAA', 'ATCACGGTCT', 'ATCCCAGCGT',
       'ATCGTACCCT', 'ATCGTACCCT', 'ATCACGGTCT', 'ATCGTACCCT',
       'ACTGTCTCTT', 'ATCCCAGCGT', 'TCTGCTAGTT', 'ATCGTGGCGT',
       'ATCCCAGCGT', 'ATCGTGGCGT', 'ATCGTGGCGT', 'ATCCCAGCGT',
       'TCTGCTAGTT', 'ATCGTACCCT', 'ATCACGGTCT', 'ATCCGCTGTA',
       'ATCACGGTCT', 'ATCACGGTCT', 'ATCTCAGTCC', 'ATCAAGCCTG',
       'ATCAGGCGTG', 'TCTGCTAGTT', 'ATCACGGTCT', 'ATCGTCTGTC',
       'CTGTCTCTTA', 'ATCCCAGCGT', 'ATCGTACCCT', 'TCTGCTAGTT',
       'ATCGTGGCGT', 'ATCGTACCCT', 'ATCAGGCGTG', 'ATCATGAAGT',
       'ATCGTGGCGT', 'ATCCATACTC', 'ATCGTGGCGT', 'ATCCTTTGCA',
       'ATCCCAGCGT', 'ATCACGGTCT', 'ATCCCAGCGT', 'ATCGTGGCGG',
       'ATCGTACCCT', 'ATCAGGCGTG', 'ATCTTAGCAA', 'ATCCCAGCGT',
       'ATCAGGCGTG', 'TCTGCTAGTT', 'ATCGTACCCT', 'ATCGTACCCT',
       'ATCGTCCACA', 'ATCGTGCCGA', 'ATCGTGGCGT', 'ATCGT

In [353]:
np.where(tt!=ttt)

(array([43, 51, 68], dtype=int64),)

In [355]:
tt[[43, 51, 68]]

array(['ATCATGAAGT', 'ATCGTGGCGG', 'ATCGTACCCA'], dtype='<U10')

In [356]:
ttt[[43, 51, 68]]

array(['ATCATGAACT', 'ATCGTGGCGT', 'ATCGTACCCT'], dtype='<U10')

In [322]:
for idx in range(0,len(tt)-1):
    class2.set_data(tt[idx+1:])
    print(class2.myfu(tt[idx]))


0
135
-1
-1
169
-1
-1
-1
163
-1
-1
60
-1
-1
32
-1
8
-1
-1
103
-1
-1
-1
0
-1
-1
-1
-1
0
-1
-1
-1
-1
-1
1
-1
-1
-1
5
-1
0
-1
0
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
0
-1
0
-1
-1
-1
-1
-1
-1
-1
-1
0
-1
-1
-1
-1
-1
0
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
27
-1
-1
-1
-1
47
-1
1
0
-1
-1
-1
0
-1
-1
-1
0
-1
-1
0
-1
-1
2
-1
0
-1
-1
-1
-1
-1
-1
18
-1
-1
2
-1
14
-1
6
-1
-1
-1
-1
-1
-1
-1
-1
0
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
0
-1
-1
26
-1
12
-1
3
-1
-1
-1
-1
-1
-1
3
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
52
0
3
1
0
-1
-1
1
-1
-1
-1
-1
-1
-1
0
-1
-1
-1
-1
1
-1
-1
-1
18
-1
-1
-1
17
-1
-1
-1
-1
-1
-1
30
-1
-1
-1
-1
16
-1
-1
-1
8
1
0
-1
0
-1
-1
-1
-1
-1
0
-1
0
-1
-1
-1
-1
-1
-1
0
-1
0
-1
-1


In [367]:
import Levenshtein as lv

ModuleNotFoundError: No module named 'Levenshtein'

In [310]:
tt

array(['TTCTATACTC', 'AACATTATCA', 'GTCTAGTAAC', 'ATCAGAATAT',
       'ATCTAAGATA', 'TTCCCCCCCC', 'ATCTAGTTAT', 'ATAGCTATTT',
       'ATCTTTTGGG', 'TTCCAGGGAA', 'AACCAAAAAT', 'ATCGCCTCAT',
       'ATCAGCGAGC', 'TTATTACTCC', 'AACCAAATCT', 'ATATATTATT',
       'TTCTACAGTT', 'ATCGGCGAAA', 'ATCGACCAAT', 'ATCCAAAAAT',
       'ATCGTGGACA', 'AAAGCCCGAG', 'TAACGGCGTG', 'AACAATAAAC',
       'CACTGTTATT', 'ATCTTAGACG', 'TTCAATACTT', 'AACGCTTATT',
       'ATCTCAGTGG', 'ATCTTTGTCA', 'ATCCTCACTC', 'ACATGAACGT',
       'ATCTTATATA'], dtype='<U10')

In [301]:
len(fvalues)

40358

In [261]:
tmp=df['BridgedBarcode'][100]

In [264]:
fvalues, finverter, fcounts = np.unique(final_mapper, return_counts=True, return_inverse=True)

In [285]:
len(fvalues)

40359

In [295]:
for idx, f in enumerate(fvalues):
    tt=(C_UMI[(fvalues==f)[finverter][inverter]])
    if (idx%100==0):
        print(idx)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 48693906 but corresponding boolean dimension is 17527016

In [279]:
helpme

array(['ATGCTGTTTCTTCAGCGTTCCCGAGATGGGTC',
       'TAATCGGATCTTCAGCGTTCCCGAGAACACGG', '', ...,
       'CCTTTCCTTCTTCAGCGTTCCCGAGACTCGGG', '',
       'CCATGAGTTCTTCAGCGTTCCCGAGATTCACA'], dtype='<U32')

In [262]:
C[np.nonzero((final_mapper==tmp)[inverter])[0]]

array(['CCATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATATTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATTTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATTTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATATTTT',
       'CTATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATTTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATATTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATATTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATGCTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATATTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATTTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATGTTTT',
       'CCATACACTCTTCAGCGTTCCCGAGATTTTTT',
       'CCA

In [289]:
C_UMI[np.nonzero((final_mapper==tmp)[inverter])[0]]

array(['CTCTTTCTCG', 'CTCCTGCGGA', 'CTCTTTATAC', 'CTCCGCATGG',
       'CTCTCGGTCT', 'TTTTTTTTTT', 'TTATTTTTTT', 'TTTTTTTTTT',
       'TTTTTTTTTT', 'CTCCGCCGTT', 'CTCCGCATGG', 'CTCTGTTCGT',
       'CTCTGGTTCT', 'CTCTGCGATT', 'TTTTTTTTTT', 'CTCCAGCTGG',
       'TTTTTTTTTT', 'TTTTTTTTTT', 'CAGCGGTTAT', 'TTTTTTTTTT',
       'TTTTTTTTTT', 'CTCTCGAGCT', 'TTTTTTTTTT', 'CTCTGTTCGT',
       'TTTTTTTTTT', 'CTCCGCATGG', 'CTCTGTCGAC', 'TTTTTTTTTT',
       'CTCCATTTAT', 'TTTTTTTTTT', 'TTTTTTTTTT', 'CTCAATGGAT',
       'CTCGTTAGTC', 'TTTGCCTTGG', 'CTCCAGCTGG', 'TTTTTTTTTT',
       'CTCTATGTGT', 'CTCCGCATGG', 'CTCGTTCACA', 'CTCTTTATAC',
       'ATTTTTTTTT', 'CTCATATCTG', 'TTTTTTTTTT', 'CTCAAACTCC',
       'TTTTTTTTTT', 'TTTTTTTTTT', 'CTCTGTCGAC', 'CCTGTCTCTT',
       'CAGCGGTTAT', 'TTTTTTTTTT', 'TTTTTTTTTT', 'CTCTGGTTCT',
       'TTTTTTTTTT', 'TTTTTTTTTT', 'CTCGTTAGTC', 'TTTTTTTTTT',
       'CTCTTTATAC', 'TTTTTTTTTT', 'CTCTGCGATT', 'TTTTTTTTTT',
       'CTCCCGGCGT', 'TTTTTTTTTT', 'TTTTTTTTTT', 'CTCGG

In [245]:
fvalues, fcounts = np.unique(final_mapper[final_mapper!=''], return_counts=True)

In [259]:
len(fvalues)

40358

In [256]:
np.sum(fcounts)

303746

In [258]:
np.sort(pairs[spatialmask,1].astype(int))[500:]

array([    5,     5,     5, ..., 32065, 40338, 48134])

In [202]:
C_UMI

AttributeError: 'list' object has no attribute 'shape'

In [180]:
C_UMI[(final_mapper==tmp)[inverter]]

TypeError: only integer scalar arrays can be converted to a scalar index

In [166]:
pairs.shape

(17527016, 2)

In [159]:
final_mapper[spatialmask]

array(['AAAAAATATCTTCAGCGTTCCCGAGAAAAAAA',
       'AAAGAAAATCTTCAGCGTTCCCGAGAAAAATG',
       'AAAAAAATTCTTCAGCGTTCCCGAGAAAATAA', ...,
       'TTTTGTTGTCTTCAGCGTTCCCGAGAACGTTA',
       'GTTTTTTTTCTTCAGCGTTCCCGAGAGTTGCC',
       'TTTATTTTTCTTCAGCGTTCCCGAGATAGCCG'], dtype='<U32')

In [156]:
final_mapper[:]=''

In [157]:
final_mapper[spatialmask]=df['BridgedBarcode'][index_mapper[spatialmask].astype(int)]

In [149]:
df['BridgedBarcode'][index_mapper[spatialmask].astype(int)]

74325    AAAAAATATCTTCAGCGTTCCCGAGAAAAAAA
22623    AAAGAAAATCTTCAGCGTTCCCGAGAAAAATG
70016    AAAAAAATTCTTCAGCGTTCCCGAGAAAATAA
15032    AAAATAAATCTTCAGCGTTCCCGAGAAAATAT
59398    GAAAAAAATCTTCAGCGTTCCCGAGAACAATA
                       ...               
27188    TTTTTTTATCTTCAGCGTTCCCGAGAGTACGG
4373     TTATTTTGTCTTCAGCGTTCCCGAGAAACATT
55802    TTTTGTTGTCTTCAGCGTTCCCGAGAACGTTA
48092    GTTTTTTTTCTTCAGCGTTCCCGAGAGTTGCC
31782    TTTATTTTTCTTCAGCGTTCCCGAGATAGCCG
Name: BridgedBarcode, Length: 303746, dtype: object

In [126]:
pairs[fivepairmask]

array([['AAAAAAAATCTTCAGCGTTCCCGAGAAAAAAA', '24'],
       ['AAAAAAAATCTTCAGCGTTCCCGAGAAAAAAC', '11'],
       ['AAAAAAAATCTTCAGCGTTCCCGAGAAAAAAT', '10'],
       ...,
       ['TTTTTTTTTCTTCAGCGTTCCCGAGAGTTGCC', '61'],
       ['TTTTTTTTTCTTCAGCGTTCCCGAGATAGCCG', '14'],
       ['TTTTTTTTTTCAGCGTTCCCGAGAAATAACAT', '5']], dtype='<U32')

In [140]:
df['BridgedBarcode'][s[0]]

'AAAAAATATCTTCAGCGTTCCCGAGAAAAAAA'

In [137]:
pairs[fivepairmask][0][0]

'AAAAAAAATCTTCAGCGTTCCCGAGAAAAAAA'

In [133]:
len(pairs[fivepairmask][1])

2

In [139]:
functions.mjollnir(df['BridgedBarcode'][s[0]], pairs[fivepairmask][0][0])

1

In [ ]:
[np.sum(commonpairs[:,1].astype(int))/len(C), np.sum(bridgedpairs[:,1].astype(int))/len(C), np.sum(fivepairs[:,1].astype(int))/len(C), np.sum(fivepairs[s>-1,1].astype(int))/len(C)]

In [40]:
[np.sum(commonpairs[:,1].astype(int))/len(C), np.sum(bridgedpairs[:,1].astype(int))/len(C), np.sum(fivepairs[:,1].astype(int))/len(C), np.sum(fivepairs[s>-1,1].astype(int))/len(C)]

[0.9341439763100744, 0.6437293744695824, 0.6067087413353001, 0.366414527374538]

We throw out 7% of the reads for not having a barcode in common with anyone else.  30% do not have something close to the bridge region.  We lose another 4% for not having more than 5 reads for the barcode.  We lose another 24% for not matching (up to a single mismatch) a known Broad barcode

In [41]:
[len(C), np.sum(commonpairs[:,1].astype(int)), np.sum(bridgedpairs[:,1].astype(int)), np.sum(fivepairs[:,1].astype(int)), np.sum(fivepairs[s>-1,1].astype(int))]

[132892946, 124141145, 85547093, 80627312, 48693906]

So out of 130M reads we are left with 49M that fulfill our requirements.

In [78]:
BridgedBarcodes=np.concatenate([df['BridgedBarcode'][[0]], df['BridgedBarcode']])
BridgedBarcodes[0]=''

In [73]:
fivepairs

array([['TCGCGCGTTCTTCAGCGTTCCCGAGACTGCAT', '48134'],
       ['CCATGATTTCTTCAGCGTTCCCGAGACGTCCC', '40338'],
       ['ACAACAAGTCTTCAGCGTTCCCGAGAACTACA', '32065'],
       ...,
       ['CGTAGAACTCTTCAACGTTCCCGAGATCTGGC', '5'],
       ['GTAATCCGTCTTCAGCGTTCCCGAGAATACAT', '5'],
       ['TGGTGTTATCTTCAGCGTTCCCGAGAGTCATT', '5']], dtype='<U32')

In [81]:
final_mapper=np.stack([fivepairs[:,0], BridgedBarcodes[s+1]], axis=1)

In [85]:
np.save('mapper.npy', final_mapper)

# Crap

DO NOT RUN THIS, THIS IS JUST IN CASE I SWITCH BACK TO BIOPYTHON FOR ALL SEQUENCES

In [ ]:
pool_class=functions.MyClass()
pool_class.set_data(df['BridgedBarcode'][0])
lst=list(bridgedpairs[:,0])
pool = Pool()
ss= (pool.map(pool_class.myfu, (lst)))
ss=np.asarray(ss)

In [17]:
df['BridgedBarcode'][0]

'TTTTNTTTTCTTCAGCGTTCCCGAGATTTTTT'

In [19]:
bridgedpairs[ss>29]

array([['CTTTTTTTTCTTCAGCGTTCCCGAGATTTTTT', '23'],
       ['TTTTGTTTTCTTCAGCGTTCCCGAGATTTTTT', '7'],
       ['TTTTTTTTCTTCAGCGTTCCCGAGATTTTTTT', '6'],
       ['GTTTTTTTTCTTCAGCGTTCCCGAGATTTTTT', '5']], dtype='<U32')

1.8M of the single barcode reads contain a bridge, this is about 36% of them

In [116]:
df.shape

(74331, 4)

In [87]:
np.sum(s>18)/len(s)

0.0

In [68]:
good_accumulated=0
all_accumulated=0
for ctr,f in enumerate(pairs[:,0]):
    pool_class.set_data(f)
    s= (pool.map(pool_class.myfu, (lst)))
    best_match=df['BridgedBarcode'][np.argmax(s)]
    b=pairwise2.align.globalms(best_match,f, 1, -1, -1, -1, penalize_end_gaps=False, one_alignment_only=True)
    all_accumulated=all_accumulated+pairs[ctr,1].astype(int)
    if(b[0][2]>29):
       #print(format_alignment(*b[0]))
       good_accumulated=good_accumulated+pairs[ctr,1].astype(int)
       print([ctr, good_accumulated/len(C), all_accumulated/len(C)])

[1, 0.0003637896370235258, 0.002147794202671501]
[3, 0.0006714785015423982, 0.0028111858375390263]
[7, 0.0009119359615650026, 0.0038514672366266427]
[8, 0.0011355035081633338, 0.0040750347832249735]
[10, 0.0013495219871012985, 0.004506801845655733]
[11, 0.0015618992200351375, 0.004719179078589573]
[12, 0.0017732917053665015, 0.004930571563920937]
[13, 0.001972479288594459, 0.005129759147148895]
[14, 0.002170652283383503, 0.0053279321419379385]
[15, 0.002368168779770897, 0.005525448638325332]
[16, 0.0025656255944854134, 0.005722905453039849]
[17, 0.002759232941299345, 0.00591651279985378]
[18, 0.0029520345855294323, 0.006109314444083868]
[19, 0.003143941004666361, 0.006301220863220796]
[20, 0.003334773153691498, 0.006492053012245934]
[21, 0.00352285994576428, 0.006680139804318715]
[22, 0.0037105886477997977, 0.006867868506354233]
[23, 0.0038956615153922765, 0.007052941373946711]
[24, 0.004080286770438176, 0.0072375666289926106]
[27, 0.004260137491653891, 0.007781953008142818]
[28, 0.004

[184, 0.024214913704031123, 0.028795422415690313]
[185, 0.024328219359988662, 0.02890872807164785]
[186, 0.024441077403399618, 0.029021586115058805]
[187, 0.024553875765137696, 0.029134384476796886]
[188, 0.024666495081857146, 0.029247003793516333]
[189, 0.024778905512721522, 0.02935941422438071]
[190, 0.02489128610274946, 0.029471794814408647]
[191, 0.02500339812524945, 0.02958390683690864]
[192, 0.025115152057712177, 0.029695660769371364]
[193, 0.025226697104319833, 0.02980720581597902]
[194, 0.025338092946745296, 0.029918601658404487]
[195, 0.025449429107497885, 0.03002993781915707]
[196, 0.02556073542741403, 0.03014124413907322]
[197, 0.025672041747330177, 0.030252550458989367]
[198, 0.025783079499718378, 0.030363588211377565]
[199, 0.02589358011705068, 0.030474088828709868]
[200, 0.02600396137103723, 0.030584470082696418]
[202, 0.026114163580005147, 0.03080493418230513]
[203, 0.026224007698935802, 0.030914778301235783]
[204, 0.026333762295357138, 0.031024532897657123]
[205, 0.0264

[369, 0.04151325865664014, 0.0477159152371041]
[370, 0.04160582493127282, 0.04780848151173677]
[372, 0.04169833152423262, 0.047993494697656375]
[373, 0.041790808276355985, 0.04808597144977974]
[374, 0.04188328502847935, 0.0481784482019031]
[375, 0.04197558273558408, 0.04827074590900783]
[376, 0.04206782076101593, 0.048362983934439684]
[377, 0.04215999910477491, 0.048455162278198655]
[378, 0.042252147607697445, 0.04854731078112119]
[380, 0.04234420658811067, 0.048731458582784076]
[381, 0.04243617604601457, 0.04882342804068798]
[382, 0.0425280858222456, 0.04891533781691901]
[383, 0.04261990607596731, 0.04900715807064072]
[384, 0.04271154728467039, 0.0490987992793438]
[385, 0.042803188493373466, 0.04919044048804688]
[386, 0.04289482970207655, 0.04928208169674996]
[387, 0.04298641122910675, 0.04937366322378016]
[388, 0.04307790323362764, 0.04946515522830105]
[389, 0.04316936539731209, 0.0495566173919855]
[390, 0.04326070819765078, 0.04964796019232419]
[391, 0.043351931634643716, 0.04973918

[567, 0.056343617231591, 0.06496749959901377]
[569, 0.056425888417652344, 0.06513207181197289]
[570, 0.05650812976287725, 0.06521431315719779]
[571, 0.05659034126726572, 0.06529652466158627]
[573, 0.056672552771654186, 0.0654609476703632]
[574, 0.05675473443520622, 0.06554312933391523]
[575, 0.05683685641708537, 0.06562525131579439]
[576, 0.05691894855812808, 0.0657073434568371]
[577, 0.0570010406991708, 0.06578943559787981]
[578, 0.05708310299937707, 0.06587149789808608]
[579, 0.05716507577707403, 0.06595347067578304]
[580, 0.057247018713934554, 0.06603541361264356]
[582, 0.05732887212828575, 0.06619915028218241]
[583, 0.05741072554263696, 0.06628100369653361]
[584, 0.05749237007113309, 0.06636264822502974]
[585, 0.05757401459962923, 0.06644429275352588]
[586, 0.05765556960561605, 0.0665258477595127]
[587, 0.057737124611602866, 0.06660740276549952]
[588, 0.057818649776753245, 0.0666889279306499]
[589, 0.05790014510106718, 0.06677042325496384]
[590, 0.057981580743708246, 0.066851858897

[756, 0.06988273297300523, 0.07985195961042788]
[757, 0.06995793188083062, 0.07992715851825327]
[758, 0.07003307110698313, 0.08000229774440577]
[759, 0.07010821033313563, 0.08007743697055829]
[760, 0.07018331971845171, 0.08015254635587435]
[761, 0.0702583694220949, 0.08022759605951754]
[762, 0.07033338928490165, 0.08030261592232431]
[763, 0.0704083196251991, 0.08037754626262174]
[764, 0.07048313060215078, 0.08045235723957342]
[765, 0.07055788189742959, 0.08052710853485223]
[766, 0.07063263319270839, 0.08060185983013105]
[767, 0.07070735464715076, 0.0806765812845734]
[768, 0.07078207610159312, 0.08075130273901578]
[769, 0.07085676771519905, 0.08082599435262171]
[770, 0.07093145932880499, 0.08090068596622764]
[772, 0.07100603157906517, 0.08104989014842086]
[773, 0.07108060382932534, 0.08112446239868104]
[774, 0.07115514623874908, 0.08119900480810477]
[775, 0.07122965880733637, 0.08127351737669207]
[776, 0.07130411169425079, 0.08134797026360649]
[778, 0.0713784750586559, 0.081496756674089

KeyboardInterrupt: 

In [49]:
np.sum(df['BridgedBarcode'].isin(pairs[:,0]))

28831

In [54]:
np.sum(np.isin(pairs[0:100000,0], df['BridgedBarcode']))

13500

In [56]:
pairs

array([['TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT', '59784'],
       ['TCGCGCGTTCTTCAGCGTTCCCGAGACTGCAT', '12191'],
       ['AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', '11920'],
       ...,
       ['CGATCTACTATTTTTTTTTTTTTTTTTTTTTT', '1'],
       ['CGATCTACTCTTCAGCGTTCCCGAGACAGATA', '1'],
       ['CTGTCTCATCTTCAGCGTTCCCGACTTTGGCG', '1']], dtype='<U32')

In [64]:
np.sum(pairs[0:500000,1].astype(int))/len(C)

0.6225061677278814

In [62]:
pairs[100000]

array(['TGCCACTTCTTCAGCGTTCCCGAGAACTGCCA', '15'], dtype='<U32')

In [40]:
bridge

'TCTTCAGCGTTCCCGAGA'

In [38]:
singles=values[counts==1]
singles[2000000:2000040]

array(['CCTGCGCCTCTTCTCTTCAGGGAACTCTCTTT',
       'CCTGCGCCTCTTCTGAATACTGTACTTCTGCA',
       'CCTGCGCCTCTTCTGCGTTCCCGAGAGGAAAA',
       'CCTGCGCCTCTTCTTAGATCTGCACAGCGAGT',
       'CCTGCGCCTCTTCTTCAGCCATTTCCCAGAAG',
       'CCTGCGCCTCTTCTTCATCCATTTCCCAGAAG',
       'CCTGCGCCTCTTGACCTTCTCCTCCTCGAACT',
       'CCTGCGCCTCTTGTTAATGTTCTTCTGAGTCA',
       'CCTGCGCCTCTTTACACAAATCGAGCTAAAAA',
       'CCTGCGCCTCTTTAGAATATGTGCTTACATTG',
       'CCTGCGCCTCTTTAGCGTTCCCGAGACAGTCC',
       'CCTGCGCCTCTTTAGCGTTCCCGAGAGGAAAT',
       'CCTGCGCCTCTTTATCGTATTCTGCAGTTCTC',
       'CCTGCGCCTCTTTCTTGATTTCCATAGATCTG',
       'CCTGCGCCTCTTTGGCCGAGAGCGGCACCGCC',
       'CCTGCGCCTCTTTGGCTTTCAAAGATCTCGTT',
       'CCTGCGCCTCTTTGGCTTTCAAAGCTCTCGTT',
       'CCTGCGCCTCTTTTCCCATCACGTTCCTTACA',
       'CCTGCGCCTCTTTTCTGCAGCCGAAGCCTCTT',
       'CCTGCGCCTCTTTTTTATTTTTTTCCCGTTAC',
       'CCTGCGCCTCTTTTTTTTTTTTGCATTTCATG',
       'CCTGCGCCTCTTTTTTTTTTTTTGCATTTCAT',
       'CCTGCGCCTCTTTTTTTTTTTTTTTTTTTTTT',
       'CCT

In [73]:
AAA='NAGTAAAATCTTCAGCGTTCCCGAGATATAGCTTCCATGTGC'
BBB='TAGTAAAATCTTCAGCGTTCCCGAGATATAGC'

In [78]:
pairwise2.align.globalms(AAA, BBB, 0.1, -1, -1, 0, penalize_end_gaps=False)

[('N-AGTAAAATCTTCAGCGTTCCCGAGATATAGCTTCCATGTGC',
  '-TAGTAAAATCTTCAGCGTTCCCGAGATATAGC----------',
  2.1000000000000005,
  0,
  43),
 ('NAGTAAAATCTTCAGCGTTCCCGAGATATAGCTTCCATGTGC',
  'TAGTAAAATCTTCAGCGTTCCCGAGATATAGC----------',
  2.1000000000000005,
  0,
  42)]

In [55]:
def check_match(seq):
    #for s in df['BridgedBarcode']:
    for idx, s in enumerate(df['BridgedBarcode']):
        aa=pairwise2.align.globalms(s,seq, 2, -1, -1, -1, one_alignment_only=True )
        if (aa[0][2]>59):
            return idx
    return -1
        

In [57]:
check_match('NAGTAAAATCTTCAGCGTTCCCGAGATATAGCTTCCATGTGC')

-1

In [58]:
rslt=df['BridgedBarcode'].apply(lambda x: pairwise2.align.globalms(x,'NAGTAAAATCTTCAGCGTTCCCGAGATATAGCTTCCATGTGC', 2, -1, -1, -1, one_alignment_only=True)[0][2])

In [62]:
b=pairwise2.align.globalms('NAGTAAAATCTTCAGCGTTCCCGAGATATAGCTTCCATGTGC', df['BridgedBarcode'][np.argmax(rslt)], 2, -1, -1, -1, one_alignment_only=True)

In [63]:
print(format_alignment(*b[0]))

NAGTAAAATCTTCAGCGTTCCCGAGATATAGCTTCCATGTGC
.||||||||||||||||||||||||  ||   || ||  |  
GAGTAAAATCTTCAGCGTTCCCGAG--AT---TT-CA--T--
  Score=51



In [45]:
a=pairwise2.align.globalms(record['sequence'], df['BridgedBarcode'][10000] , 2, -1, -1, -1, one_alignment_only=True)

In [47]:
a[0]

IndexError: list index out of range

# Experimental

In [25]:
pool_class=functions.MyClass()
pool_class.set_data(pairs[0,0])
lst=list(df['BridgedBarcode'])
pool = Pool()


In [68]:
good_accumulated=0
all_accumulated=0
for ctr,f in enumerate(pairs[:,0]):
    pool_class.set_data(f)
    s= (pool.map(pool_class.myfu, (lst)))
    best_match=df['BridgedBarcode'][np.argmax(s)]
    b=pairwise2.align.globalms(best_match,f, 1, -1, -1, -1, penalize_end_gaps=False, one_alignment_only=True)
    all_accumulated=all_accumulated+pairs[ctr,1].astype(int)
    if(b[0][2]>29):
       #print(format_alignment(*b[0]))
       good_accumulated=good_accumulated+pairs[ctr,1].astype(int)
       print([ctr, good_accumulated/len(C), all_accumulated/len(C)])

[1, 0.0003637896370235258, 0.002147794202671501]
[3, 0.0006714785015423982, 0.0028111858375390263]
[7, 0.0009119359615650026, 0.0038514672366266427]
[8, 0.0011355035081633338, 0.0040750347832249735]
[10, 0.0013495219871012985, 0.004506801845655733]
[11, 0.0015618992200351375, 0.004719179078589573]
[12, 0.0017732917053665015, 0.004930571563920937]
[13, 0.001972479288594459, 0.005129759147148895]
[14, 0.002170652283383503, 0.0053279321419379385]
[15, 0.002368168779770897, 0.005525448638325332]
[16, 0.0025656255944854134, 0.005722905453039849]
[17, 0.002759232941299345, 0.00591651279985378]
[18, 0.0029520345855294323, 0.006109314444083868]
[19, 0.003143941004666361, 0.006301220863220796]
[20, 0.003334773153691498, 0.006492053012245934]
[21, 0.00352285994576428, 0.006680139804318715]
[22, 0.0037105886477997977, 0.006867868506354233]
[23, 0.0038956615153922765, 0.007052941373946711]
[24, 0.004080286770438176, 0.0072375666289926106]
[27, 0.004260137491653891, 0.007781953008142818]
[28, 0.004

[184, 0.024214913704031123, 0.028795422415690313]
[185, 0.024328219359988662, 0.02890872807164785]
[186, 0.024441077403399618, 0.029021586115058805]
[187, 0.024553875765137696, 0.029134384476796886]
[188, 0.024666495081857146, 0.029247003793516333]
[189, 0.024778905512721522, 0.02935941422438071]
[190, 0.02489128610274946, 0.029471794814408647]
[191, 0.02500339812524945, 0.02958390683690864]
[192, 0.025115152057712177, 0.029695660769371364]
[193, 0.025226697104319833, 0.02980720581597902]
[194, 0.025338092946745296, 0.029918601658404487]
[195, 0.025449429107497885, 0.03002993781915707]
[196, 0.02556073542741403, 0.03014124413907322]
[197, 0.025672041747330177, 0.030252550458989367]
[198, 0.025783079499718378, 0.030363588211377565]
[199, 0.02589358011705068, 0.030474088828709868]
[200, 0.02600396137103723, 0.030584470082696418]
[202, 0.026114163580005147, 0.03080493418230513]
[203, 0.026224007698935802, 0.030914778301235783]
[204, 0.026333762295357138, 0.031024532897657123]
[205, 0.0264

[369, 0.04151325865664014, 0.0477159152371041]
[370, 0.04160582493127282, 0.04780848151173677]
[372, 0.04169833152423262, 0.047993494697656375]
[373, 0.041790808276355985, 0.04808597144977974]
[374, 0.04188328502847935, 0.0481784482019031]
[375, 0.04197558273558408, 0.04827074590900783]
[376, 0.04206782076101593, 0.048362983934439684]
[377, 0.04215999910477491, 0.048455162278198655]
[378, 0.042252147607697445, 0.04854731078112119]
[380, 0.04234420658811067, 0.048731458582784076]
[381, 0.04243617604601457, 0.04882342804068798]
[382, 0.0425280858222456, 0.04891533781691901]
[383, 0.04261990607596731, 0.04900715807064072]
[384, 0.04271154728467039, 0.0490987992793438]
[385, 0.042803188493373466, 0.04919044048804688]
[386, 0.04289482970207655, 0.04928208169674996]
[387, 0.04298641122910675, 0.04937366322378016]
[388, 0.04307790323362764, 0.04946515522830105]
[389, 0.04316936539731209, 0.0495566173919855]
[390, 0.04326070819765078, 0.04964796019232419]
[391, 0.043351931634643716, 0.04973918

[567, 0.056343617231591, 0.06496749959901377]
[569, 0.056425888417652344, 0.06513207181197289]
[570, 0.05650812976287725, 0.06521431315719779]
[571, 0.05659034126726572, 0.06529652466158627]
[573, 0.056672552771654186, 0.0654609476703632]
[574, 0.05675473443520622, 0.06554312933391523]
[575, 0.05683685641708537, 0.06562525131579439]
[576, 0.05691894855812808, 0.0657073434568371]
[577, 0.0570010406991708, 0.06578943559787981]
[578, 0.05708310299937707, 0.06587149789808608]
[579, 0.05716507577707403, 0.06595347067578304]
[580, 0.057247018713934554, 0.06603541361264356]
[582, 0.05732887212828575, 0.06619915028218241]
[583, 0.05741072554263696, 0.06628100369653361]
[584, 0.05749237007113309, 0.06636264822502974]
[585, 0.05757401459962923, 0.06644429275352588]
[586, 0.05765556960561605, 0.0665258477595127]
[587, 0.057737124611602866, 0.06660740276549952]
[588, 0.057818649776753245, 0.0666889279306499]
[589, 0.05790014510106718, 0.06677042325496384]
[590, 0.057981580743708246, 0.066851858897

[756, 0.06988273297300523, 0.07985195961042788]
[757, 0.06995793188083062, 0.07992715851825327]
[758, 0.07003307110698313, 0.08000229774440577]
[759, 0.07010821033313563, 0.08007743697055829]
[760, 0.07018331971845171, 0.08015254635587435]
[761, 0.0702583694220949, 0.08022759605951754]
[762, 0.07033338928490165, 0.08030261592232431]
[763, 0.0704083196251991, 0.08037754626262174]
[764, 0.07048313060215078, 0.08045235723957342]
[765, 0.07055788189742959, 0.08052710853485223]
[766, 0.07063263319270839, 0.08060185983013105]
[767, 0.07070735464715076, 0.0806765812845734]
[768, 0.07078207610159312, 0.08075130273901578]
[769, 0.07085676771519905, 0.08082599435262171]
[770, 0.07093145932880499, 0.08090068596622764]
[772, 0.07100603157906517, 0.08104989014842086]
[773, 0.07108060382932534, 0.08112446239868104]
[774, 0.07115514623874908, 0.08119900480810477]
[775, 0.07122965880733637, 0.08127351737669207]
[776, 0.07130411169425079, 0.08134797026360649]
[778, 0.0713784750586559, 0.081496756674089

KeyboardInterrupt: 

End of parallel one mismatch finding

In [53]:
thecounts=pairs[:,1].astype(int)

In [62]:
np.sum(thecounts[thecounts==1])/np.sum(thecounts)

0.14842966328346183

In [63]:
np.sum(thecounts==1)/len(thecounts)

0.6008662107501651

Only 15% of the NGS sequenced barcodes show up only one time, but they represent 60% of the things to search

This is good stuff, but it searches each barcode individually, this takes a while, what if we turn all barcodes into one long string

In [72]:
the_long_chain="".join([s for s in df['BridgedBarcode']])


In [73]:
pairwise2.align.globalms(the_long_chain,pairs[0,0], 1, -1, -1, -1, penalize_end_gaps=False, one_alignment_only=True)

KeyboardInterrupt: 